<a href="https://colab.research.google.com/github/660710078/TopGun2025/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install ultralytics opencv-python-headless matplotlib seaborn

from google.colab import drive
import cv2
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt
from pathlib import Path
import yaml

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
base_path = '/content/drive/MyDrive/drone_dataset'
yaml_path = f'{base_path}/data.yaml'

# ตรวจสอบโครงสร้างไฟล์
!ls -la {base_path}

# อ่าน yaml file
with open(yaml_path, 'r') as f:
    data_config = yaml.safe_load(f)
print(data_config)

total 9
-rw------- 1 root root  113 Oct 27 04:37 data.yaml
drwx------ 4 root root 4096 Oct 27 03:59 train
drwx------ 4 root root 4096 Oct 27 03:59 valid
{'path': '/content/drive/MyDrive/drone_dataset', 'train': 'train/images', 'val': 'valid/images', 'names': {0: 'drone'}, 'nc': 1}


In [13]:
def enhance_image(image):
    """
    ปรับปรุงคุณภาพภาพก่อนส่งเข้า YOLO
    """
    # 1. แปลงเป็น RGB
    if len(image.shape) == 2: # 2 มิติ - 3
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

    # 2. ปรับ Contrast (CLAHE)
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab) # แยก L, A, B ออกเป็น 3 channels
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8)) #แบ่งภาพเป็นตาราง 8x8 แล้วปรับแต่ละช่องแยก ส่วนมืดสว่างขึ้น, ส่วนสว่างไม่สว่างจนเกิน
    l = clahe.apply(l)  # ปรับแค่ channel L
    enhanced = cv2.merge([l,a,b])  # รวมกลับ
    enhanced = cv2.cvtColor(enhanced, cv2.COLOR_LAB2RGB)

    # 3. Denoise หา pixel ที่คล้ายกันในบริเวณใกล้เคียง เฉลี่ยค่าของ pixel ที่คล้ายกัน
    denoised = cv2.fastNlMeansDenoisingColored(enhanced, None, 10, 10, 7, 21)

    # 4. Sharpen เพิ่มความคมชัด
    kernel = np.array([[-1,-1,-1],
                       [-1, 9,-1],
                       [-1,-1,-1]])
    sharpened = cv2.filter2D(denoised, -1, kernel)

    return sharpened # ภาพที่มี contrast ดี, noise น้อย, ขอบคมชัด → YOLO ตรวจจับได้ง่ายขึ้น

def detect_sky_region(image):
    """
    ตัดส่วนท้องฟ้าออกเพื่อลด false positives
    """
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    # Sky มักมีค่า Saturation ต่ำ และ Value สูง
    lower_sky = np.array([0, 0, 150])
    upper_sky = np.array([180, 50, 255])
    mask = cv2.inRange(hsv, lower_sky, upper_sky) #สร้าง binary mask
    return mask

def visualize_preprocessing(original, processed):
    """
    แสดงผลการ preprocessing
    """
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    axes[0].imshow(original)
    axes[0].set_title('Original')
    axes[0].axis('off')

    axes[1].imshow(processed)
    axes[1].set_title('Processed')
    axes[1].axis('off')
    plt.show()

In [1]:
# เลือก model size (n, s, m, l, x)
model = YOLO('yolov8n.pt')  # nano สำหรับความเร็ว

# Train
results = model.train(
    data=yaml_path,
    epochs=100,
    imgsz=640,
    batch=16,
    name='drone_detection',
    patience=20,
    save=True,
    # device=0,  # GPU

    # Augmentation
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=10,
    translate=0.1,
    scale=0.5,
    flipud=0.5,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.1
)

NameError: name 'YOLO' is not defined

In [ ]:
# โหลด trained model
model = YOLO(f'{base_path}/runs/detect/drone_detection/weights/best.pt') #x (xlarge) - ใหญ่สุด, ช้าสุด, แม่นมาก

def detect_drone_with_processing(image_path, conf_threshold=0.25):
    """
    Detection พร้อม image processing
    """
    # อ่านภาพ
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    original = image.copy()

    # 1. Pre-processing
    processed = enhance_image(image)

    # 2. Detection
    results = model.predict(
        source=processed,
        conf=conf_threshold,
        iou=0.45,
        imgsz=640,
        #device=0
    )

    # 3. Post-processing (กรอง false positives)
    filtered_boxes = []
    for box in results[0].boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        conf = box.conf[0].cpu().numpy()

        # เช็คว่าไม่อยู่ในพื้นที่ท้องฟ้าเกินไป
        center_y = (y1 + y2) / 2
        if center_y > image.shape[0] * 0.2:  # ไม่อยู่บนสุดของภาพ
            filtered_boxes.append(box)

    # 4. Visualize
    fig, axes = plt.subplots(1, 2, figsize=(15, 7))

    # Original
    axes[0].imshow(original)
    axes[0].set_title('Original Image')
    axes[0].axis('off')

    # Detection result
    result_img = results[0].plot()
    axes[1].imshow(result_img)
    axes[1].set_title(f'Detected Drones: {len(filtered_boxes)}')
    axes[1].axis('off')

    plt.tight_layout()
    plt.show()

    return results, filtered_boxes

# ทดสอบ
test_image = f'{base_path}/test/images/your_image.jpg'
results, detections = detect_drone_with_processing(test_image, conf_threshold=0.3)

In [ ]:
def process_dataset(folder_path, output_path, conf_threshold=0.25):
    """
    ประมวลผลทั้ง folder
    """
    from pathlib import Path
    import os

    Path(output_path).mkdir(parents=True, exist_ok=True)

    image_files = list(Path(folder_path).glob('*.jpg')) + \
                  list(Path(folder_path).glob('*.png'))

    results_summary = []

    for img_path in image_files:
        print(f"Processing: {img_path.name}")

        # Detect
        results, detections = detect_drone_with_processing(
            str(img_path),
            conf_threshold
        )

        # Save result
        result_img = results[0].plot()
        output_file = os.path.join(output_path, f"result_{img_path.name}")
        cv2.imwrite(output_file, cv2.cvtColor(result_img, cv2.COLOR_RGB2BGR))

        results_summary.append({
            'image': img_path.name,
            'detections': len(detections)
        })

    return results_summary

# ใช้งาน
summary = process_dataset(
    f'{base_path}/test/images',
    f'{base_path}/results',
    conf_threshold=0.3
)

print("\n=== Summary ===")
for item in summary:
    print(f"{item['image']}: {item['detections']} drones")

In [ ]:
# ประเมินผล
metrics = model.val(data=yaml_path)

print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")

accuracy = (metrics.box.mp + metrics.box.mr) / 2
print(f"Accuracy:  {accuracy:.4f}")